In [384]:
!pip install jdc

In [385]:
import pandas as pd
import numpy as np
import jdc

In [386]:
class MDP():
  def __init__(self, num_state, num_actions, filename, gamma):
    self.num_states = num_state
    self.num_actions = num_actions
    self.filename = filename
    self.gamma = gamma

    self.stateDic = {}
    self.rewardArr = [[] for i in range(self.num_states)]
    self.rewardActionArr = [[] for i in range(self.num_states)]
    

    self.read_from_file()

In [387]:
%%add_to MDP
def read_from_file(self):
  file = open(self.filename, "r")

  data_obj = []
  
  for line in file:
    data_obj.append(line)

  file.close()

  #print(data_obj)

  for i, obj in enumerate(data_obj):
    obj = obj.rstrip()
    obj = obj.split("(")
    obj = "".join(obj)
    obj = obj.split(")")
    obj = "".join(obj)
    obj = obj.split(" ")
    data_obj[i] = obj

  for x, obj in enumerate(data_obj):
    action = {}
    for a in range(self.num_actions):
      i = a+1
      action['a'+str(i)] = []
    self.stateDic[obj[0]] = {"reward":int(obj[1]), "actions": action}
    self.rewardArr[x].append(int(obj[1]))
    

  # self.stateDic["s1"]["actions"]["a1"].append({"s1": 0.509})
  # self.stateDic["s1"]["actions"]["a1"].append({"s2": 0.491})

  # print(data_obj[2])

  for obj in data_obj:
    idx = 3
    for action in obj[2::3]:
      self.stateDic[obj[0]]["actions"][action].append({obj[idx]: float(obj[idx+1])})
      idx += 3
  
  #print(self.stateDic)
  #print(self.rewardArr)

In [388]:
%%add_to MDP
def Bellman_equation(self):
  """
  E.g., [a1] ==> J^2(s1) = (r(s1) = 5) + ((gamma = 0.8)*( ((Prob(a1, s1) = 0.509) * J^1(s1) = 5)  +  ((Prob(a1, s2) = 0.491) * J^1(s2) = 10))
        [a2] ==> J^2(s1) = (r(s1) = 5) + ((gamma = 0.8)*( ((Prob(a1, s1) = 0.31) * J^1(s1) = 5)  +  ((Prob(a1, s3) = 0.69) * J^1(s3) = -5))
        J^2(s1) = max(a1, a2)

  """

  # states = {"s1": 0, "s2": 1, "s3": 2} # 0: "s1", 1: "s2", 2: "s3"}
  states = {}
  for i in range(self.num_states):
    states["s"+str(i+1)] = i
  #print(states) 

  for k in range(1,21): # 20):
    # print("k = ", k)
    for state, st_idx in states.items():
      reward = self.stateDic[state]["reward"]
      # print(state)
      # print(reward)
      tmp_j_si = []
      rew_idx = 0
      highest_reward = -999
      best_action = ""
      for action in self.stateDic[state]["actions"]:
        #print(action)
        num_state_trans = len(self.stateDic[state]["actions"][action])
        j_sum = 0
        for trans_idx in range(num_state_trans):
          s_lst = list(self.stateDic[state]["actions"][action][trans_idx])[0] # get the states inside the action list
          # print("{} -> r = {}".format( s_lst, self.rewardArr[states[s_lst]][k-1] ) )

          j_sum += ( self.stateDic[state]["actions"][action][trans_idx][s_lst] * self.rewardArr[states[s_lst]][k-1] )
        j_si = reward + (self.gamma * j_sum)
        j_si = round(j_si, 3)
        if (j_si > highest_reward):
          highest_reward = j_si
          best_action = action
        #print(best_action)
        #print(j_si)
        tmp_j_si.append(j_si)
      self.rewardArr[st_idx].append(max(tmp_j_si))
      self.rewardActionArr[st_idx].append(best_action)


In [389]:
n_state = 3
n_actions = 2
filename = 'test1.in'

# n_state = 10 # 3
# n_actions = 3 # 2
# filename = 'test2.in'
discount = 0.8
mdp = MDP(n_state, n_actions, filename, discount)
mdp.Bellman_equation()

st = {}

for i in range(mdp.num_states):
  st[i] = "s"+str(i+1)
#print(st)

# for i, val in st.items():
#   print("{} ==> {}". format(val, mdp.rewardArr[i]))

for i in range(0,20):
  print("After iteration " + str(i+1) + ":", end=" ")
  for s in range(mdp.num_states):
    if(s+1 == mdp.num_states):
      print("(s" + str(s+1) + " " + str(mdp.rewardActionArr[s][i]) + " " + str(mdp.rewardArr[s][i]) + ")")
    else:
      print("(s" + str(s+1) + " " + str(mdp.rewardActionArr[s][i]) + " " + str(mdp.rewardArr[s][i]) + ")", end=" ")



After iteration 1: (s1 a1 5) (s2 a1 10) (s3 a2 -5)
After iteration 2: (s1 a1 10.964) (s2 a1 12.8) (s3 a2 2.2)
After iteration 3: (s1 a1 14.492) (s2 a1 17.108) (s3 a2 4.946)
After iteration 4: (s1 a1 17.621) (s2 a1 19.93) (s3 a2 8.268)
After iteration 5: (s1 a1 20.004) (s2 a1 22.406) (s3 a2 10.575)
After iteration 6: (s1 a1 21.947) (s2 a1 24.317) (s3 a2 12.54)
After iteration 7: (s1 a1 23.489) (s2 a1 25.869) (s3 a2 14.074)
After iteration 8: (s1 a1 24.726) (s2 a1 27.103) (s3 a2 15.314)
After iteration 9: (s1 a1 25.714) (s2 a1 28.092) (s3 a2 16.302)
After iteration 10: (s1 a1 26.505) (s2 a1 28.883) (s3 a2 17.093)
After iteration 11: (s1 a1 27.138) (s2 a1 29.516) (s3 a2 17.726)
After iteration 12: (s1 a1 27.644) (s2 a1 30.022) (s3 a2 18.232)
After iteration 13: (s1 a1 28.049) (s2 a1 30.427) (s3 a2 18.637)
After iteration 14: (s1 a1 28.373) (s2 a1 30.751) (s3 a2 18.961)
After iteration 15: (s1 a1 28.632) (s2 a1 31.01) (s3 a2 19.22)
After iteration 16: (s1 a1 28.84) (s2 a1 31.217) (s3 a2 19